In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from random import randint
from PIL import Image,ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
from random import shuffle


transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
    transforms.RandomRotation(randint(1,360)),
    transforms.RandomHorizontalFlip(),

])
train_path = "./Final Dataset/train"
test_path = "./Final Dataset/test"
val_path = "./Final Dataset/val"
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False
train_data = torchvision.datasets.ImageFolder(root=train_path,transform=transforms,is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_path,transform=transforms,is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_path,transform=transforms,is_valid_file=check_image)
batch_size = 64
num_classes = 10
train_loader = DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)
val_loader = DataLoader(dataset=val_data,batch_size=batch_size/2,shuffle=True)

In [9]:
if torch.cuda.is_available:
    device = torch.device('cuda')
    torch.cuda.empty_cache()
else:
    device = torch.device('cpu')

In [18]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.fc = nn.Linear(1024, num_classes)
model.to(device)

Using cache found in C:\Users\manje/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\torchvision\models\googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [21]:
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.001,weight_decay=1e-4)

criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=20, gamma=0.1, last_epoch=-1,verbose=True)

c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:
from tqdm import tqdm
import numpy as np
num_epochs=100
device='cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

train_losses=[]
val_loss_min=np.Inf

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    valid_loss = 0.0
    model.train()
    for _,(images,labels,_) in enumerate(tqdm(train_loader,desc=f'Epoch {epoch+1}')):
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)
    model.eval()
    for images,labels,_ in val_loader:
        images,labels=images.to(device),labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        valid_loss += loss.item()*images.size(0)
    
    train_loss = train_loss/len(train_loader.dataset)
    val_loss = valid_loss/len(val_loader.dataset)
    train_losses.append(train_loss)
        
    print(f'\nTraining Loss: {train_loss:.4f} \tValidation Loss: {val_loss:.4f}')
    if val_loss <= val_loss_min:
        torch.save(model.state_dict(), 'resnet50.pt')
        val_loss_min = val_loss